In [2]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
kdtree = None
radius = 1.3

def init_kdtree(tree):
    global kdtree
    kdtree = tree

def find_plane_directions(indices, points, radius=2):
    normals = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        neighbors = points[neighbor_indices]
        mean = np.mean(neighbors, axis=0)
        norm = neighbors - mean
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eigh(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_vec = eig_vec[:, sorted_idx]
        plane_direction = eig_vec[:, 2]
        normals.append(plane_direction)
    return normals

def calculate_normal_standard_deviation(indices, points, normals, radius=2):
    standard_deviations = []

    for idx in indices:
        # **Step 1: Neighbor Query**
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)

        # **Step 2: Retrieve Neighbor Normals (Optimized)**
        neighbor_normals = normals[neighbor_indices]  # Direct indexing (no conversion)

        if len(neighbor_normals) == 0:
            standard_deviations.append(0)  
            continue

        # **Step 3: Align Normals**
        reference_normal = neighbor_normals[0]
        dot_products = np.dot(neighbor_normals, reference_normal)
        aligned_normals = neighbor_normals * np.sign(dot_products)[:, np.newaxis]

        # **Step 4: Standard Deviation Calculation**
        std_dev = np.std(aligned_normals, axis=0)
        variation_measure = np.sum(std_dev)  

        standard_deviations.append(variation_measure)

    return standard_deviations  

if __name__ == "__main__": 
    dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
    pcd = o3d.io.read_point_cloud(dataname)
    pcd = util.preProcessCloud(pcd)
    myarray = np.asarray(pcd.points)
    indices = np.arange(len(myarray))
    kdtree = cKDTree(myarray)
    num_chunks = 16
    chunk_size = len(myarray) // num_chunks
    chunked_indices = [indices[i:i + chunk_size] for i in range(0, len(indices), chunk_size)]

    # First pass: Compute normals
    start_time_first_pca = time.time()
    with multiprocessing.Pool(processes=4, initializer=init_kdtree, initargs=(kdtree,)) as pool:
        normals_chunks = pool.starmap(find_plane_directions, [(chunk_indices, myarray, radius) for chunk_indices in chunked_indices])
    first_pca_duration = time.time() - start_time_first_pca
    print(f"First PCA time: {first_pca_duration:.2f} seconds")

    # Flatten normals to a single array in order
    all_normals = np.vstack(normals_chunks)

    # Second pass: Compute standard deviation-based variation
    start_time_standard_deviations = time.time()
    with multiprocessing.Pool(processes=4, initializer=init_kdtree, initargs=(kdtree,)) as pool:
        standard_deviation_chunks = pool.starmap(calculate_normal_standard_deviation, [(chunk_indices, myarray, all_normals, radius) for chunk_indices in chunked_indices])
    standard_deviations_duration = time.time() - start_time_standard_deviations
    print(f"Second PCA time: {standard_deviations_duration:.2f} seconds")

    # Flatten standard deviations to maintain order
    standard_deviations = np.hstack(standard_deviation_chunks)

    # Normalize variation values
    max_variation = np.max(standard_deviations) if len(standard_deviations) > 0 else 1
    standard_deviations /= max_variation

    # Print a few results to verify
    print("First 5 normals:")
    print(all_normals[:5])
    print("First 5 normalized standard deviations:")
    print(standard_deviations[:5])


First PCA time: 119.69 seconds
Second PCA time: 77.77 seconds
First 5 normals:
[[ 1.63706886e-02 -4.28153068e-03  9.99856824e-01]
 [ 9.81775780e-03 -4.16497540e-03  9.99943131e-01]
 [ 5.19251839e-03 -5.49079091e-03  9.99971444e-01]
 [ 1.97441588e-03 -3.88094137e-03  9.99990520e-01]
 [ 3.98232665e-04 -1.84545855e-03  9.99998218e-01]]
First 5 normalized standard deviations:
[0.02728182 0.02846811 0.02966687 0.03067699 0.03043064]


In [6]:
def compute_local_variation_trends(indices, points, standard_deviations, radius=2):
    """
    Computes local variation trends by sorting neighborhood points and detecting increase-decrease patterns.
    """
    bend_candidates = []
    for idx in indices:
        neighbor_indices = kdtree.query_ball_point(points[idx], radius)
        if len(neighbor_indices) < 5:  # Ignore small neighborhoods
            continue

        # Sort neighbors by Euclidean distance to the center point
        neighbor_points = points[neighbor_indices]
        neighbor_stds = standard_deviations[neighbor_indices]
        distances = np.linalg.norm(neighbor_points - points[idx], axis=1)
        sorted_indices = np.argsort(distances)

        # Get sorted std deviation values
        sorted_stds = neighbor_stds[sorted_indices]

        # Compute first derivative (local differences)
        first_derivative = np.diff(sorted_stds)

        # Compute second derivative (change of change)
        second_derivative = np.diff(first_derivative)

        # Identify bends where first derivative changes from positive to negative (concave-up peaks)
        bend_mask = (np.roll(first_derivative, 1) > 0) & (first_derivative < 0)

        # Identify bends where the first derivative changes from positive to negative
        bend_mask = (np.roll(first_derivative, 1) > 0) & (first_derivative < 0)

        # Trim sorted_indices to match bend_mask length
        bend_points = np.array(neighbor_indices)[sorted_indices[:-1]][bend_mask]
        bend_candidates.extend(bend_points)

    return np.unique(bend_candidates)  # Return unique bend candidate indices

In [7]:
bend_candidates = compute_local_variation_trends(indices, myarray, standard_deviations, radius)

In [8]:
def visualize_bend_candidates(pcd, bend_candidates):
    """
    Visualizes detected bend candidates by coloring them red.
    """
    colors = np.ones((len(np.asarray(pcd.points)), 3)) * 0.6  # Default gray color

    # Set bend candidate points to red
    colors[bend_candidates] = [1, 0, 0]  # RGB (Red)

    # Apply colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Create Open3D visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window("Bend Candidates Visualization")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()

# Call visualization function
visualize_bend_candidates(pcd, bend_candidates)